In [4]:
import mysql.connector
import mysql.connector.errorcode
import pandas as pd
import numpy as np

cnx = "";
unique_meters: pd.DataFrame = None;
def connect_to_database():
    global cnx
    cnx = mysql.connector.connect(
            user="guilhermec",
            password="6B&54V$Z7rkD",
            host="iot-laager.c3wepnzq2di2.us-east-1.rds.amazonaws.com",
            database="DLAAGERIOT"
    )


In [18]:
import pandas as pd

connect_to_database()
if cnx and cnx.is_connected():
    global unique_meters
    with cnx.cursor() as cursor:
        cursor.execute(
            """
                select bu.name, r.rgi, r.hydrometer_code,
                r.status, concat('https://iot.laager.com.br/so/images/full/', soi.hash_value)  from residences r 
                inner join commercial_services cs on(r.commercial_service_id = cs.id)
                inner join business_units bu on (cs.business_unit_id = bu.id)
                inner join service_orders so on (so.residence_id = r.id)
                inner join service_order_images soi on (soi.service_order_id = so.id)
                where bu.id = 762 and r.status = 'ACTIVATED' and soi.stage = 'POS'
            """)
        
        data = cursor.fetchall()
        meters = pd.DataFrame(data, columns=["business_unit", "rgi", "meter", "status", "photo_link"])
        display(meters)
        meters.to_excel("data.xlsx", index=False)


    cnx.close();
else:
    print("Não conectado")

,business_unit,rgi,meter,status,photo_link
0,Águas de Joinville,00227855-3,G23CN8000011,ACTIVATED,https://iot.laager.com.br/so/images/full/6bc6f...
1,Águas de Joinville,00227855-3,G23CN8000011,ACTIVATED,https://iot.laager.com.br/so/images/full/6b880...
2,Águas de Joinville,00227855-3,G23CN8000011,ACTIVATED,https://iot.laager.com.br/so/images/full/6c9ad...
3,Águas de Joinville,00228158-9,K23CN8000057,ACTIVATED,https://iot.laager.com.br/so/images/full/58ede...
4,Águas de Joinville,00228158-9,K23CN8000057,ACTIVATED,https://iot.laager.com.br/so/images/full/82f36...
...,...,...,...,...,...
3381,Águas de Joinville,01314032-9,B16B004313,ACTIVATED,https://iot.laager.com.br/so/images/full/993c9...
3382,Águas de Joinville,01314032-9,B16B004313,ACTIVATED,https://iot.laager.com.br/so/images/full/bfc3c...
3383,Águas de Joinville,01314032-9,B16B004313,ACTIVATED,https://iot.laager.com.br/so/images/full/e914b...
3384,Águas de Joinville,01314032-9,B16B004313,ACTIVATED,https://iot.laager.com.br/so/images/full/c0a5c...


In [19]:
data = pd.read_excel("./data.xlsx")
data.sort_values(by=["rgi", "photo_link"], inplace=True)

In [20]:
import requests
photo_links = data["photo_link"].to_numpy();
first_url = photo_links[0]


In [27]:
import os

for unique_rgi in data["rgi"].unique():
    if unique_rgi in ("00227855-3", "01314032-9"):
        subdata = data[data["rgi"]==unique_rgi]
        dir_to_create = fr"./{unique_rgi}"
        os.mkdir(dir_to_create)
        for i, photo_link in enumerate(subdata["photo_link"].unique()):
            req = requests.request("GET", url=photo_link);
            with open(fr"{dir_to_create}/image{i}.jpg", "wb") as handler:
                handler.write(req.content);